# Setup

#### Load Packages

In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

# import packages
import numpy as np
import matplotlib.pyplot as plt

# import modules
import Bargaining as brg
from figures import *
from checks import *

#### Compile C++ files

In [2]:
# compile c++ files
model = brg.HouseholdModelClass(par={'do_cpp':True,'num_Ctot':100})
try:
    model.link_to_cpp(force_compile=True)
except:
    model.cpp.delink()
    model.link_to_cpp(force_compile=True)

terminal:

e:\AHJ\projects\guide\HouseholdBargainingGuide>cd /d "C:/Program Files/Microsoft Visual Studio/2022/Community/VC/Auxiliary/Build/" 

C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Auxiliary\Build>call vcvarsall.bat x64 
**********************************************************************
** Visual Studio 2022 Developer Command Prompt v17.7.6
** Copyright (c) 2022 Microsoft Corporation
**********************************************************************
[vcvarsall.bat] Environment initialized for: 'x64'
solve.cpp
e:\AHJ\projects\guide\HouseholdBargainingGuide\cppfuncs\index.cpp(18): warning C5208: unnamed class used in typedef name cannot declare members other than non-static data members, member enumerations, or member classes
C:\Program Files\Microsoft Visual Studio\2022\Community\VC\Tools\MSVC\14.37.32822\include\algorithm(7912): warning C4244: 'argument': conversion from 'double' to 'int', possible loss of data
C:\Program Files\Microsoft Visual Studio\2022

AttributeError: 'types.SimpleNamespace' object has no attribute 'delink'

## Model Settings

#### Specify Models

In [ ]:
# Default settings
T = 40
do_load = False
do_save = False
threads = 50

# adjusted model
parameters = {
#     'rho_w': 3.0,
#     'alpha2_w': 3.0,
       'max_A': 40.0,
       'max_A_pd': 40.0,
       'num_A': 400,
       'num_A_pd': 400,
       
       'sigma_love':0.0,
       'div_A_share':0.3,
       'R':1.03,
}

# Model settings
specs = {
       # 'VFI':    
       #        {'latexname':'', 
       #        'load': do_load,
       #        'save': do_save,
       #        'par':{'T':T,
       #               'do_cpp':True,
       #               'threads':threads,
                     
       #               'div_A_share':0.3,
       #               'sigma_love':0.0,
       #               'R':1.2,
       #               'do_egm':False,
       #               'max_A': 15.0,
       #               }
       #        },
              
       # 'EGM':    
       #        {'latexname':'', 
       #        'load': do_load,
       #        'save': do_save,
       #        'par':{'T':T,
       #               'do_cpp':True,
       #               'threads':threads,
                     
       #               'div_A_share':0.3,
       #               'sigma_love':0.0,
       #               'R':1.2,
       #               'do_egm':True,
       #               'max_A': 15.0,
       #               'analytic_marg_u_single':False,
       #               'analytic_inv_marg_u_single':False,
       #               'do_upper_env':True,
       #               }
       #        },

       'weird VFI':    
              {'latexname':'', 
              'load': do_load,
              'save': do_save,
              'par':{'T':T,
                     'do_cpp':True,
                     'threads':threads,

                     'do_egm':False,

                     **parameters
                     }
              },

       # 'weird EGM':    
       #        {'latexname':'', 
       #        'load': do_load,
       #        'save': do_save,
       #        'par':{'T':T,
       #               'do_cpp':True,
       #               'threads':threads,
                     
       #               'div_A_share':0.3,
       #               'sigma_love':0.0,
       #               'R':1.2,
       #               'do_egm':True,
       #               'max_A': 15.0,
       #               'analytic_marg_u_single':False,
       #               'analytic_inv_marg_u_single':False,
       #               'do_upper_env':False,

       #               **parameters
       #               }
       #        },
       'weird EGM with envelope':    
              {'latexname':'', 
              'load': do_load,
              'save': do_save,
              'par':{'T':T,
                     'do_cpp':True,
                     'threads':threads,

                     'do_egm':True,
                     'analytic_marg_u_single':False,
                     'analytic_inv_marg_u_single':False,
                     'do_upper_env':True,
                     'marg_V_couple_finite_diff': True,
                     **parameters
                     }
              },

}

In [ ]:
# get the first variable of specs
name = list(specs.keys())[0]

In [ ]:
# Make table
print_specs_table(specs)

#### Solve/Load models

In [ ]:
# solve different models
models = {}
for name,spec in specs.items():
        #unpack
        par = spec['par']
        do_load = spec['load']
        do_save = spec['save']
        
        if do_load:
            print(f'loading {name}...')
        else:
            print(f'solving {name}...')
        
        # setup model
        models[name] = brg.HouseholdModelClass(name=name, par=spec['par'], load=do_load)
        models[name].spec = spec
        
        # link to cpp
        try:
            models[name].link_to_cpp(force_compile=False)
        except:
            models[name].cpp.delink()
            models[name].link_to_cpp(force_compile=False)
        
        # solve
        if not do_load:
            models[name].solve()
        
        # save model
        if do_save:
            models[name].save()
        
# Save model names
model_names  = list(models.keys())

## Compare solutions

In [ ]:
# Choose index
t  = 0 # non-monotonous bc of bargaining (when div_A_share = 0.3, marg_V_couple_finite_diff = False)
iP = 10 # 14
iL = 5
iA = 149

# t = 0 # non-monotonous bc of divorce (when div_A_share = 0.8)
# iP = 15
# iL = 20
# iA = 30


idx = (t,iP,iL,iA)

Endogenous grids

In [ ]:
par = models['weird EGM with envelope'].par
sol = models['weird EGM with envelope'].sol

# make a big figure
fig = plt.figure(figsize=(12,8))

for t in range(par.T):
    plt.plot(sol.M_pd[t,iP,iL,:], sol.C_tot_pd[t,iP,iL,:], label=f'Endo_{t}')
    plt.plot(par.grid_A*par.R + par.inc_w + par.inc_m, sol.C_tot_couple[t,iP,iL,:], linestyle='--', label=f'Exo_{t}')
    
    # Draw dot at end of asset grid
    plt.scatter(par.grid_A_pd[-1]+sol.C_tot_pd[t,iP,iL,-1], sol.C_tot_pd[t,iP,iL,-1], color='black', s=5.0)
    # plt.plot(par.grid_A_pd[-4:]+sol.C_tot_pd[t,iP,iL,-4:], sol.C_tot_pd[t,iP,iL,-4:], color='black')
    
    # annotate t at end of line 
    if t == par.T-1:
        plt.annotate(f't={t}', (par.grid_A[-1]*par.R + par.inc_w + par.inc_m, sol.C_tot_couple[t,iP,iL,-1]), fontsize=10)
    else:
        plt.annotate(f't={t}', (sol.M_pd[t,iP,iL,-1], sol.C_tot_pd[t,iP,iL,-1]), fontsize=10)
    
# axis labels
plt.xlabel('M')
plt.ylabel('C')

# axis limits
# plt.xlim([0, 32])
# plt.ylim([2.0, 2.8])

# Draw vertical line at end of asset grid
plt.axvline(par.grid_A[-1], linestyle='--', color='black', alpha=0.5)
plt.axvline(par.grid_A[-1]*par.R + par.inc_w + par.inc_m, linestyle='--', color='black', alpha=0.5)


In [ ]:
for t in range(par.T).__reversed__():
    print(f't={t}: {sol.C_tot_couple[t,iP,iL,-4:]}')

In [ ]:
for t in range(par.T).__reversed__():
    print(f't={t}: {sol.C_tot_pd[t,iP,iL,-4:]}')

In [ ]:
M = par.grid_A*par.R + par.inc_w + par.inc_m
M

In [ ]:
# is_sorted = lambda a: np.all(a[:-1] <= a[1:])

# for t in range(par.T):
#     for iP in range(par.num_power):
#         for iL in range(par.num_love):
#             print(f't={t}, iP={iP}, iL={iL}: {is_sorted(sol.M_pd[t,iP,iL,:])}')

V couple

In [ ]:
par = models['weird EGM with envelope'].par
sol = models['weird EGM with envelope'].sol

# make a big figure
fig = plt.figure(figsize=(12,8))

V_couple = np.zeros((par.T,par.num_A))
for t in range(par.T):
    V_couple[t,:] = par.grid_power[iP]*sol.Vm_remain_couple[t,iP,iL,:] + (1-par.grid_power[iP])*sol.Vw_remain_couple[t,iP,iL,:]
    
    M = par.grid_A*par.R + par.inc_w + par.inc_m
    plt.plot(sol.M_pd[t,iP,iL,:], sol.V_couple_pd[t,iP,iL,:], label=f'Endo_{t}')
    plt.plot(M, V_couple[t,:], linestyle='--', label=f'Exo_{t}')
    
    # annotate t at end of line 
    if t == par.T-1:
        plt.annotate(f't={t}', (par.grid_A[-1]*par.R + par.inc_w + par.inc_m, V_couple[t,-1]), fontsize=10)
    else:
        plt.annotate(f't={t}', (sol.M_pd[t,iP,iL,-1], sol.V_couple_pd[t,iP,iL,-1]), fontsize=10)
    
# axis labels
plt.xlabel('M')
plt.ylabel('V')

# axis limits
# plt.xlim([0, 32])
# plt.ylim([-15, -13])

# Draw vertical line at end of asset grid
plt.axvline(par.grid_A[-1], linestyle='--', color='black', alpha=0.5)
plt.axvline(par.grid_A[-1]*par.R + par.inc_w + par.inc_m, linestyle='--', color='black', alpha=0.5)


In [ ]:
sol.Vw_remain_couple[t,iP,iL,:]
iP

In [ ]:
for t in range(par.T).__reversed__():
    print(f't={t}: {sol.power[t,iP,iL,-4:]*sol.Vm_couple[t,iP,iL,-4:] + (1-sol.power[t,iP,iL,-4:])*sol.Vw_couple[t,iP,iL,-4:]}')

In [ ]:
for t in range(par.T).__reversed__():
    print(f't={t}: {sol.V_couple_pd[t,iP,iL,-4:]}')

marg V couple

In [ ]:
par = models['weird EGM with envelope'].par
sol = models['weird EGM with envelope'].sol

# make a big figure
fig = plt.figure(figsize=(12,8))

V_couple = np.zeros((par.T,par.num_A))
for t in range(par.T):

    M = par.grid_A*par.R + par.inc_w + par.inc_m
    plt.plot(M, sol.marg_V_couple[t,iP,iL,:], label=f'Endo_{t}')
    
    # annotate t at end of line 
    plt.annotate(f't={t}', (M[-1], sol.marg_V_couple[t,iP,iL,-1]), fontsize=10)
    
# axis labels
plt.xlabel('M')
plt.ylabel('marg_V')

# axis limits
# plt.xlim([0, 32])
# plt.ylim([2.6, 2.8])

# Draw vertical line at end of asset grid
plt.axvline(par.grid_A[-1], linestyle='--', color='black', alpha=0.5)
plt.axvline(par.grid_A[-1]*par.R + par.inc_w + par.inc_m, linestyle='--', color='black', alpha=0.5)


In [ ]:
for t in range(par.T).__reversed__():
    print(f't={t}: {sol.marg_V_couple[t,iP,iL,-4:]}')

marg U

In [ ]:
par = models['weird EGM with envelope'].par
sol = models['weird EGM with envelope'].sol

# make a big figure
fig = plt.figure(figsize=(12,8))

for t in range(par.T):

    plt.plot(sol.M_pd[t,iP,iL,:], sol.EmargU_pd[t,iP,iL,:], label=f'Endo_{t}')
    
    # annotate t at end of line 

    plt.annotate(f't={t}', (sol.M_pd[t,iP,iL,-1], sol.EmargU_pd[t,iP,iL,-1]), fontsize=10)
    
# axis labels
plt.xlabel('M')
plt.ylabel('margU')

# axis limits
# plt.xlim([0, 32])
# plt.ylim([2.6, 2.8])

# Draw vertical line at end of asset grid
plt.axvline(par.grid_A[-1], linestyle='--', color='black', alpha=0.5)
plt.axvline(par.grid_A[-1]*par.R + par.inc_w + par.inc_m, linestyle='--', color='black', alpha=0.5)


In [ ]:
for t in range(par.T).__reversed__():
    print(f't={t}: {sol.EmargU_pd[t,iP,iL,-4:]}')

In [ ]:
# par = models['weird EGM with envelope'].par
# sol = models['weird EGM with envelope'].sol

# # make a big figure
# fig = plt.figure(figsize=(12,8))

# for t in range(par.T):
#     plt.plot(sol.M_pd[t,iP,iL,:], sol.EmargU_pd[t,iP,iL,:], label=f'Endo_{t}')
    
#     plt.annotate(f't={t}', (sol.M_pd[t,iP,iL,-1], sol.EmargU_pd[t,iP,iL,-1]), fontsize=10)

# # axis labels
# plt.xlabel('M')
# plt.ylabel('margU')

# # Draw vertical line at end of asset grid
# plt.axvline(par.grid_A[-1], linestyle='--', color='black', alpha=0.5)
# plt.axvline(par.grid_A[-1]*par.R + par.inc_w + par.inc_m, linestyle='--', color='black', alpha=0.5)

# Other

Bargaining

In [ ]:
model_plot(models, plot_surplus, [''], t, iP, iL, 20)

In [ ]:
# par = models['weird EGM with envelope'].par
# sol = models['weird EGM with envelope'].sol

# Sw = sol.Vw_remain_couple[t,:,iL,iA]-sol.Vw_single[t,iA]
# Sm = sol.Vm_remain_couple[t,:,iL,iA]-sol.Vm_single[t,iA]

# # plt.plot(par.grid_power, Sm)
# # plt.plot(par.grid_power, Sw)

# print('marg_V_remain\n', sol.marg_V_remain_couple[t+1,:,iL,iA])
# print('V_couple_pd\n', sol.V_couple_pd[:,iL,iA])


# print('Vw_remain\n', sol.Vw_remain_couple[t,:,iL,iA])
# print('Vm_remain\n', sol.Vm_remain_couple[t,:,iL,iA])

# print('C_tot_pd\n', sol.C_tot_pd[iP,iL,:])
# print('grid_A_pd\n', par.grid_A_pd)
# print('M_pd\n', sol.M_pd[iP,iL,:])
# print('M\n', par.R * par.grid_A + par.inc_w + par.inc_m)


Value functions, woman

In [ ]:
model_plot(models, plot_var_over_assets, ['Vw_couple', 'Vw_remain_couple', 'Vw_single'], idx=idx, asset_grid=False, shared_legend=True, subtitles=model_names)

Value functions, man

In [ ]:
model_plot(models, plot_var_over_assets, ['Vm_couple', 'Vm_remain_couple', 'Vm_single'], idx=idx, asset_grid=False, shared_legend=True, subtitles=model_names)

Marginal values

In [ ]:
model_plot(models, plot_var_over_assets, ['marg_V_couple', 'marg_V_remain_couple'], idx=idx, asset_grid=False, shared_legend=True, subtitles=model_names)

Bargaining power

In [ ]:
model_plot(models, plot_var_over_assets, ['power'], idx=idx, asset_grid=False, shared_legend=True, subtitles=model_names)

In [ ]:
#Next period bargaining power
idx_lead = (t+1,iP,iL,iA)
model_plot(models, plot_var_over_assets, ['power'], idx=idx_lead, asset_grid=False, shared_legend=True, subtitles=model_names)

Total consumption

In [ ]:
model_plot(models, plot_var_over_assets, ['C_tot_couple', 'C_tot_remain_couple'], idx=idx, asset_grid=False, shared_legend=True, subtitles=model_names)

In [ ]:
VFI = models['weird VFI']

V_remain_couple = (VFI.par.grid_power.reshape(-1,1))*VFI.sol.Vw_remain_couple[t,:,iL,:] + (1-VFI.par.grid_power.reshape(-1,1))*VFI.sol.Vm_remain_couple[t,:,iL,:]
V_couple = (VFI.par.grid_power.reshape(-1,1))*VFI.sol.Vw_couple[t,:,iL,:] + (1-VFI.par.grid_power.reshape(-1,1))*VFI.sol.Vm_couple[t,:,iL,:] 

sol_power = VFI.sol.power[t,:,iL,:]
power = VFI.par.grid_power.reshape(-1,1)
V_couple_sol = (sol_power)*VFI.sol.Vw_couple[t,:,iL,:] + (1-sol_power)*VFI.sol.Vm_couple[t,:,iL,:]

## So what's going on with weird EGM?

In [ ]:
model = models['weird EGM with envelope']
sol = model.sol
par = model.par

In period t+1, bargaining happens, meaning that the value function is shifted - all sorts of weird stuff can then happen with the marginal value

In [ ]:
%matplotlib inline
power = model.par.grid_power[iP]
V_couple = power*sol.Vw_couple + (1-power)*sol.Vm_couple
V_remain_couple = power*sol.Vw_remain_couple + (1-power)*sol.Vm_remain_couple

ub = 100

# plot with two subfigures
fig, ax = plt.subplots(1,2, figsize=(12,4))
ax[0].plot(par.grid_A[:ub], V_couple[t+1,iP,iL,:ub], label='couple')
ax[0].plot(par.grid_A[:ub], V_remain_couple[t+1,iP,iL,:ub], label='remain couple')
ax[0].set_title('V (time t+1)')
ax[0].set_xlabel('A')   

ax[1].plot(par.grid_A[:ub], sol.marg_V_couple[t+1,iP,iL,:ub], label='couple')
ax[1].plot(par.grid_A[:ub], sol.marg_V_remain_couple[t+1,iP,iL,:ub], label='remain couple')
ax[1].set_title('marginal V (time t+1)')
ax[1].set_xlabel('A')

lines, labels = ax[0].get_legend_handles_labels()
fig.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, 0.05), bbox_transform=fig.transFigure, ncol=3);

In [ ]:
par.grid_A_pd

In [ ]:
plt.scatter(par.grid_A_pd, sol.C_tot_pd[iP,iL])
plt.ylabel('C_tot')
plt.xlabel('A pd')
plt.title("Consumption")

...which then makes the endogenous grid non-monotonous as well.

In [ ]:
plt.scatter(sol.M_pd[iP,iL],par.grid_A_pd)
plt.xlabel('M')
plt.ylabel('A')
plt.title("Endogenous grid")
# endogenous grid is crazy due to bargaining shifting the value function

Let's pretend everything is okay and do upper envelope anyway...

In [ ]:
# A bunch of code doing upper envelope, written off from consav
size = par.num_A_pd

C_now = np.zeros(size)
C_now[:] = sol.C_tot_pd[iP, iL,:] 

M_now = np.zeros(size)
M_now[:] = model.sol.M_pd[iP, iL,:]

V_now = np.zeros(size)
V_now[:] = model.sol.V_couple_pd[iP, iL,:]

Vm_next = np.zeros((par.num_love,par.num_A))
Vm_next[:,:] = model.sol.Vm_couple[t+1,iP,:,:]

Vw_next = np.zeros((par.num_love,par.num_A))
Vw_next[:,:] = model.sol.Vw_couple[t+1,iP,:,:]

value = lambda C,M: model.value_of_choice_couple(C, t, M, iL, iP, model.par.grid_power[iP], Vw_next, Vm_next)[0]

# Jeppe's algorithm
Na = par.num_A_pd
Nm = par.num_A

# post decision grid
grid_a = par.grid_A_pd
grid_m = par.grid_A

# endogenous grids
m_vec = sol.M_pd[iP, iL]
c_vec = sol.C_tot_pd[iP, iL]
w_vec = sol.V_couple_pd[iP, iL]

# common grid
c_ast_vec = np.zeros(Nm)
v_ast_vec = np.zeros(Nm) - np.inf


# constraint
# binding if common m is smaller than the smallest m implied by EGM step (m_vec[0])
im = 0
while im < Nm and grid_m[im] < m_vec[0]:
    # consume all
    c_ast_vec[im] = grid_m[im]

    # value of choice
    v_ast_vec[im] = value(c_ast_vec[im], grid_m[im])

    im +=1

for ia in range(Na-1):
    a_low = grid_a[ia]
    a_high = grid_a[ia+1]

    # w_low = w_vec[ia]
    # w_high = w_vec[ia+1]

    if a_low>a_high:
        continue

    # w_slope = (w_high-w_low)/(a_high-a_low)

    # m interval and c slope
    m_low = m_vec[ia]
    m_high = m_vec[ia+1]

    c_low = c_vec[ia]
    c_high = c_vec[ia+1]

    c_slope = (c_high-c_low)/(m_high-m_low)

    # loop through common grid
    for im in range(Nm):
        # current m
        m = grid_m[im]

        # interpolate?
        interp = (m>m_low) and (m<m_high)
        extrap_above = ia==Na-2 and m>m_vec[Na-1] # return to this

        # interpolation or extrapolation
        if interp or extrap_above:

            # implied guess:
            c_guess = c_low + c_slope*(m-m_low)
            a_guess = m - c_guess

            # implied post-decision value fnuction
            # w = w_low + w_slope*(a_guess-a_low)

            # value of choice
            v_guess = value(c_guess, m)

            # update
            if v_guess > v_ast_vec[im]:
                c_ast_vec[im] = c_guess
                v_ast_vec[im] = v_guess

plt.plot(grid_m, c_ast_vec, linewidth=5, label='upper envelope')
plt.plot(m_vec,c_vec, label='EGM consumption')
plt.plot(grid_m, grid_m, color='black', linestyle='--', alpha=0.5, label='45 degree')
plt.legend()
plt.xlabel('M')
plt.title('Consumption');

Resulting in...

In [ ]:
plt.plot(sol.M_pd[iP,iL],sol.C_tot_pd[iP, iL])

# scatter plot of the same thing with numbers on the grid points
plt.scatter(sol.M_pd[iP,iL],sol.C_tot_pd[iP, iL], label="Endogenous grid")
for i in range(0, len(sol.M_pd[iP,iL])):
    if i % 5 == 0:
        plt.text(sol.M_pd[iP,iL][i],sol.C_tot_pd[iP, iL][i], f"{par.grid_A_pd[i]:5.2f}", fontsize=8)

plt.plot(par.grid_A,par.grid_A, color='black', linestyle='--', alpha=0.5)

plt.xlabel('M')
plt.ylabel('C_tot')

plt.plot(par.grid_A, sol.C_tot_remain_couple[t,iP,iL,:], label=model.name)
plt.plot(models['weird VFI'].par.grid_A, models['weird VFI'].sol.C_tot_remain_couple[t,iP,iL,:], label=models['weird VFI'].name)
plt.legend();

Questions:
* How do we handle the non-monotenous marginal value?
* Given that we figure out how the non-monotonous marginal value works, how do we do upper envelope here? I don't think the savings function is necessarily (weakly) increasing in this problem, so we need to figure out how to deal with that.
    * I also kinda think that the value function is not the right fnuction to use for upper envelope here...
* How do we impose the budget constraint? Right now, it is very clearly not done right (let go of budget constraint -> decrease consumption)